In [31]:
using DataFrames
using Plotly
using JLD

In [32]:
names = [:DPC,:MCC,:DATE,:TIKR,:ORTC,:ORSC,:TCOR,:TCSH,:CNSH,:MCSH,:AWSH,:f0t9,:f10t29,
        :f30t59,:f60t299,:ft30m,:ARSP,:AESP,:PISH,:PIAM,:PIAT,:AQSH,:AQAT,:OQSH,:OQAM,:OQAT]
raw = readtable("DATA/VIRTU1216.txt",header = false, separator = '|', names = names);


In [33]:
Plotly.plot(histogram(x = raw[:ORTC]))

In [34]:
#I am looking at marketable limit and market orders

mktable = raw[(raw[:ORTC].==11).|(raw[:ORTC].==12),:];
mktable[mktable[:TIKR].=="AAPL",:];

In [35]:
size(mktable)

(12254, 26)

In [36]:
mktable = mktable[completecases(mktable),:];
size(mktable)

(12254, 26)

In [37]:
#adding executed shares to feature space for weighting
mktable[:EXSH] = mktable[:TCSH] .- mktable[:CNSH]; #total covered shares - cancelled shares

In [38]:
#initializing the securities array

securities = unique(mktable[:TIKR]);
len = length(securities);
uind = Int.(zeros(len))

for i = 1:len
    uind[i] = findfirst(mktable[:TIKR],securities[i])
end

bysec = mktable[uind,:];

#since I am consolidating by security I clear out all of the following features

delete!(bysec,:DPC); #designated participant code
delete!(bysec,:MCC); #market center code
delete!(bysec,:DATE); #date
delete!(bysec,:ORTC); #order type code
delete!(bysec,:ORSC); #order size code

#this will initially contain dummy values based on first appearance of securities, but this saves the hassle of
#initializing the DataFrame by hand and declaring all the right datatypes


In [39]:
#Checking if indexing is preserved for next steps 
if sum(isequal.(bysec[:TIKR],securities)) - len != 0
    mktable = NA #break the whole thing otherwise
end

In [40]:
summable = [:TCOR,:TCSH,:CNSH,:MCSH,:AWSH,:f0t9,:f10t29,:f30t59,:f60t299,:ft30m,:PISH,:AQSH,:OQSH,:EXSH]
# need weighting: [:ARSP,:AESP,:PIAM,:PIAT,:AQAT,:OQAM,:OQAT]


#here we go!

for i = 1:len
    
    tempdf = mktable[mktable[:TIKR] .== securities[i],:]
    
    for s in summable
        bysec[s][i] = sum(tempdf[s])
    end
    
    bysec[:ARSP][i] = sum(tempdf[:ARSP] .* tempdf[:EXSH])/bysec[:EXSH][i]
    bysec[:AESP][i] = sum(tempdf[:AESP] .* tempdf[:EXSH])/bysec[:EXSH][i]
    bysec[:PIAM][i] = sum(tempdf[:PIAM] .* tempdf[:PISH])/bysec[:PISH][i]
    bysec[:PIAT][i] = sum(tempdf[:PIAT] .* tempdf[:PISH])/bysec[:PISH][i]    
    bysec[:AQAT][i] = sum(tempdf[:AQAT] .* tempdf[:AQSH])/bysec[:AQSH][i]        
    bysec[:OQAM][i] = sum(tempdf[:OQAM] .* tempdf[:OQSH])/bysec[:OQSH][i]
    bysec[:OQAT][i] = sum(tempdf[:OQAT] .* tempdf[:OQSH])/bysec[:OQSH][i]

end

In [41]:
bysec[(bysec[:TIKR] .== "AAPL") .| (bysec[:TIKR] .== "TSLA"),:]

,TIKR,TCOR,TCSH,CNSH,MCSH,AWSH,f0t9,f10t29,f30t59,f60t299,ft30m,ARSP,AESP,PISH,PIAM,PIAT,AQSH,AQAT,OQSH,OQAM,OQAT,EXSH
1,AAPL,2753,574607,570748,3859,0,3859,0,0,0,0,0.0036620367970976934,0.09780671158331174,800,0.006625,0.0,1409,0.0,1650,0.0829,0.0,3859
2,TSLA,1357,206291,114635,91656,0,91656,0,0,0,0,-0.021850634982979834,0.13836018154839835,33990,0.003232621359223301,0.0,42754,0.0,14912,0.0659828057939914,0.0,91656


In [42]:
# average shares per order
bysec[:SHPO] = bysec[:TCSH]./bysec[:TCOR]
plot(histogram(x = bysec[:SHPO]))

In [43]:
crsp = readtable("DATA/CRSP1216.csv");
size(completecases(crsp))[1]/size(crsp)[1]
crsp = crsp[.!isna.(crsp[:TICKER]),:];
sum(completecases(crsp))

398

In [44]:
secs = intersect(unique(crsp[:TICKER]),securities);
size(secs)

(5584,)

In [45]:
bysec = bysec[indexin(bysec[:TIKR],secs).>0,:]
size(bysec)

(5584, 23)

In [46]:
crsp = crsp[indexin(crsp[:TICKER],secs).>0,:]
size(unique(crsp[:TICKER]))

(5584,)

In [47]:
len = length(secs);
uind = Int.(zeros(len))

for i = 1:len
    uind[i] = findfirst(crsp[:TICKER],secs[i])
end

crsp_bysec = crsp[uind,:];
size(crsp_bysec)

(5584, 17)

In [48]:
sum(isequal.(bysec[:TIKR],crsp_bysec[:TICKER])) - len
#need to get these in the same order

-5584

In [49]:
sec_ind = Int.(zeros(len))

for i = 1:len
    sec_ind[i] = findfirst(crsp_bysec[:TICKER],bysec[:TIKR][i])
end

crsp_bysec = crsp_bysec[sec_ind,:]
secs = bysec[:TIKR]

#check
sum(isequal.(bysec[:TIKR],crsp_bysec[:TICKER])) - len

0

In [50]:
delete!(crsp_bysec,:date)
crsp_bysec[1,:]

,PERMNO,EXCHCD,NCUSIP,TICKER,PRIMEXCH,PERMCO,CUSIP,HSICIG,MMCNT,PRC,VOL,BID,ASK,SHROUT,OPENPRC,NUMTRD
1,87432,1,00846U10,A,N,36364,00846U10,NA,NA,43.21,2823273,43.21,43.22,324385,44.08,NA


In [51]:
take_avg = [:MMCNT,:PRC,:VOL,:BID,:ASK,:SHROUT,:OPENPRC,:NUMTRD]

for s in take_avg
    crsp_bysec[s][isna.(crsp_bysec[s])] = -7777
    crsp_bysec[s] = float(crsp_bysec[s])
    crsp_bysec[s][crsp_bysec[s].==-7777] = NaN
end


for i = 1:len
    tempdf = crsp[crsp[:TICKER] .== secs[i],:]
    for s in take_avg

        if sum(.!isna.(tempdf[s])) >= 3. #if we have 3 or more values available
            crsp_bysec[s][i] = mean(tempdf[s][.!isna.(tempdf[s])])
        else
            crsp_bysec[s][i] = NaN
        end

    end

end
    


In [52]:
xdf = DataFrame()
PESP = bysec[:AESP]./crsp_bysec[:PRC]; #PERCENTAGE EFFECTIVE SPREAD
xdf[:DVOL] = crsp_bysec[:VOL].*crsp_bysec[:PRC];
plot(xdf[:DVOL],PESP,mode= "markers")


In [53]:
xdf[:MMCNT] = crsp_bysec[:MMCNT]
plot(xdf[:MMCNT],PESP,mode = "markers")

In [54]:
sum(!isnan.(crsp_bysec[:MMCNT]))

2161

In [55]:
sum(!isnan.(xdf[:DVOL]))

5576

In [56]:
xdf[:MCAP] = crsp_bysec[:SHROUT].*crsp_bysec[:PRC];
plot(xdf[:MCAP],xdf[:DVOL],mode = "markers")


In [57]:
xdf[:SPO] = bysec[:TCSH]./bysec[:TCOR]
xdf[:MCSHP] = bysec[:MCSH]./bysec[:TCSH]
xdf[:AWSHP] = bysec[:AWSH]./bysec[:TCSH]
xdf[:CNSHP] = bysec[:CNSH]./bysec[:TCSH]
xdf[:FAST] = bysec[:f0t9]./(bysec[:EXSH])
xdf[:SLOW] = (bysec[:f60t299].+bysec[:ft30m])./bysec[:EXSH]
xdf[:PVOL] = xdf[:DVOL]./xdf[:MCAP]
xdf[:MMCNT] = crsp_bysec[:MMCNT]
xdf[:Y] = PESP
xdf[:PCPRC] = (crsp_bysec[:PRC] .- crsp_bysec[:OPENPRC])./crsp_bysec[:PRC]
xdf[:NUMTRD] = crsp_bysec[:NUMTRD]

sum(bysec[:TCOR])
delete!(xdf,:FAST)
delete!(xdf,:SLOW)
delete!(xdf,:MCSHP)
delete!(xdf,:AWSHP)
delete!(xdf,:Y)
delete!(xdf,:PVOL)
delete!(xdf,:PCPRC)
xdf[10,:]

,DVOL,MMCNT,MCAP,SPO,CNSHP,NUMTRD
1,5.209798157823129e6,38.27272727272727,1.739407714285714e6,129.90384615384616,0.6210214655810511,1415.1904761904761


In [58]:
size(raw)

(27562, 26)

In [59]:

X = Array(xdf); #column1: VOL, column2: MMCNT, column3: MCAP
Y = Array(PESP); # percentage effective spread
save("X1.jld","X",X)
save("Y1.jld","Y",Y)